# Azure AI Agent service - Bing custom search

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingCustomSearchTool, MessageRole
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 17-Jun-2025 11:20:28


## 1. Grounding with Bing Custom Search

### Connect to AI Foundry

In [5]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to custom bing grounding service, the config only has ARXIV.ORG in it !!

In [6]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("BING_CUSTOM_CONNECTION_ID")

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=conn_id, instance_name="first-config")

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"
agent = project_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-bing-grounding-agent",
        instructions=instructions,
        tools=bing_custom_tool.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")

bing_custom_tool
bing_custom_tool.definitions


Created agent, ID: asst_JCxzqPPvFkFkGVYsHBT3dOZV


[{'type': 'bing_custom_search', 'bing_custom_search': {'search_configurations': [{'connection_id': '/subscriptions/7a28b21e-0d3e-4435-a686-d92889d4ee96/resourceGroups/AI-FOUNDRY-RG/providers/Microsoft.CognitiveServices/accounts/aq-ai-foundry-Sweden-Central/projects/firstProject/connections/aqbingcustomsearch002', 'instance_name': 'first-config', 'market': '', 'set_lang': '', 'count': 5, 'freshness': ''}]}}]

### Create THREAD

In [8]:
# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_IJA14Pdh1xl8kzp0Lj3zj6ay


### Example #1

In [9]:
query = "what is are the latest news on AI?"

### Create first message in the thread

In [10]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_tRswqy0osOLWS9CWOUFhRHZj


### Create first RUN

In [11]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


### Check status of the RUN and tool calls

In [12]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage


Step step_VrZN8NcKWi8XBhKafp0OWVqQ status: completed
Step step_ugOV40dN6Jbv6lrZRiVJ2LVO status: completed
  Tool calls:
    Tool Call ID: call_qIRvwIfxoXLZ8qCxhxgBVDNB
    Type: bing_custom_search



{'prompt_tokens': 11189, 'completion_tokens': 485, 'total_tokens': 11674, 'prompt_token_details': {'cached_tokens': 0}}

### Get response

In [13]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: Some of the latest news and updates on AI as of June 2025 include:

- The release of the Artificial Intelligence Index Report 2025, which discusses AI's accelerating impact on society, the economy, and global governance. New topics in this year’s report include advancements in AI hardware, unique estimates of AI inference costs, and trends in AI publishing and patents. There's also more on how businesses are adopting responsible AI and on AI's growing role in science and medicine【3:0†[2504.07139] Artificial Intelligence Index Report 2025 - arXiv.org†https://arxiv.org/abs/2504.07139】.
- Recent AI research has focused on self-supervised video models (V-JEPA 2), new approaches for AI agents in strategic decision-making, value alignment in agentic AI systems, and the increased speed and efficiency in planning algorithms for AI (such as fast Monte Carlo tree diffusion)【3:1†Artificial Intelligence - arXiv.org†https://arxiv.org/list/cs.AI/recent】.
- Studies in human-AI interac

### Example #2

In [14]:
query = "find out about the latest iPhone"

In [15]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_MmUSQWArezepwBXV7UMDLvub


In [16]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [17]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_sfoyQgom0mtDJukONsfCBkvo status: completed
Step step_PXWxgBZYQv0S6gkPoEEKFzZD status: completed
  Tool calls:
    Tool Call ID: call_mAVnTujfg60DIfmm6W2ROsRX
    Type: bing_custom_search



{'prompt_tokens': 9348, 'completion_tokens': 171, 'total_tokens': 9519, 'prompt_token_details': {'cached_tokens': 0}}

In [18]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: The latest iPhone, expected to be called the iPhone 17, is rumored to be launching in September 2025. Leaks and reports suggest it will feature a thinner and lighter design, improved battery life, and possibly a new type of display technology for better brightness and efficiency. Enhanced AI integration is also expected, especially for on-device tasks such as photography, voice control, and productivity. The devices may offer up to 2TB of storage, and some sources speculate about advanced camera features and new colors. However, as of mid-June 2025, Apple has not officially confirmed these details, and the full specifications will be revealed closer to the launch date.

If you want specifics on hardware or features, let me know!


### Example #3

In [19]:
query = "What is the weather informations for Mexico city? Show the report and use emojis and be funny"

In [20]:
# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_y7rxaMopQxTEFqBrLWQ4k4P2


In [21]:
# Create and process agent run in thread with tools
run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: RunStatus.COMPLETED


In [22]:
# Fetch run steps to get the details of the agent run
run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])

    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")

print()  # add an extra newline between steps

run.usage

Step step_VhDL0Q1MQyLeirgnKISfS5wX status: completed
Step step_x3Pc3ThlAdpKxXtDsCxYUY75 status: completed
  Tool calls:
    Tool Call ID: call_PMZjCo8R2rEHD1NeP275FIi2
    Type: bing_custom_search



{'prompt_tokens': 4335, 'completion_tokens': 262, 'total_tokens': 4597, 'prompt_token_details': {'cached_tokens': 0}}

In [23]:
# Print the Agent's response message with optional citation
response_message = project_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Agent response: 🌞 Mexico City Weather Report! 🌦️

Hello from the land of tacos and traffic! 🌮🚗 As of today, Mexico City is dancing between mild sun and a chance of afternoon showers (because, of course, those umbrellas want to feel useful too ☔). Expect daytime highs around 24°C (75°F) 🌡️—warm enough that the quesadillas melt just right, but not so hot that you need to fry an egg on the pavement (yet 🥚🔥)! Nights might cool down to around 13°C (55°F), so bring a light jacket unless you prefer to shiver salsa-style! 💃

Humidity is high enough to frizz your hair into an Aztec pyramid, but at least the cactuses are happy. 🌵 If the wind picks up, beware: your sombrero might just take flight! 🎩💨

Sneak Preview: With the rainy season in full swing, don’t be surprised if the afternoon skies suddenly decide they’re in a dramatic telenovela and burst into tears for an hour. ☔🌈

Stay dry, stay cool, and may your coffee always be caliente! ☕😄


### Post processing

In [24]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_JCxzqPPvFkFkGVYsHBT3dOZV, Name: my-bing-grounding-agent, Model: gpt-4.1, Instructions: You are an AI Agent that can do some web search using Bing


In [26]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")

In [27]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")